Rough Draft for the take home final. 

In [ ]:
!pip install xmltodict

In [ ]:
rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]
ratesForBase = [r for r in rates if r != "USD" and r != "EUR" and r != "GBP"]

Code Given by Professor Powers in the Prompt. 

In [ ]:
import requests
import xmltodict
import json
import random

# URL of the XML data
date = "2011-05-04"
base = random.choice(ratesForBase)
url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
print(url)
# Fetch the XML data
response = requests.get(url)
response.raise_for_status()  # Ensure we notice bad responses

# Parse the XML data to a Python dictionary
data_dict = xmltodict.parse(response.text)

# Convert the dictionary to a JSON string
json_data = json.dumps(data_dict, indent=4)

# Print the JSON data
print(json_data)

# Optionally, write the JSON data to a file
with open(f"{date}_exchange_rates_{base}.json", "w") as json_file:
    json_file.write(json_data)

Quest 1 Attempt 1: 

In [ ]:
import os
import requests 
import xmltodict
import json
import random
from datetime import datetime, timedelta
import threading
import time

rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP",
         "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW", "KWD", "LYD",
         "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR",
         "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]

rates_for_base = [r for r in rates if r not in ["USD", "EUR", "GBP"]]
base = random.choice(rates_for_base)


start = datetime(2011, 5, 4)
end = datetime.today()
dates = []
while start <= end:
    dates.append(start.strftime("%Y-%m-%d"))
    start += timedelta(days=1)

folder = "data_" + base
os.makedirs(folder, exist_ok=True)

def download(date):
    path = f"{folder}/{date}.json"
    if os.path.exists(path):
        return
    url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base}&format_type=xml"
    try:
        r = requests.get(url, timeout=10)
        d = xmltodict.parse(r.text)
        with open(path, "w") as f:
            json.dump(d, f)
        print("Saved", date)
    except:
        print("Failed", date)

def run_all():
    i = 0
    while i < len(dates):
        group = []
        for j in range(10):
            if i + j < len(dates):
                t = threading.Thread(target=download, args=(dates[i + j],))
                t.start()
                group.append(t)
        for t in group:
            t.join()
        i += 10

print("Starting", base)
run_all()
print("Done")

Tried to separate above code:

In [ ]:
import threading
from downloader import download_data
from utils import get_all_dates, pick_random_base, already_downloaded

dates = get_all_dates("2011-05-04")
base = pick_random_base()

threads = []

def download_for_data(date):
    if not already_downloaded(date, base):
        print("Getting:", date)
        download_data(date, base)
    else:
        print("Already have:", date)

for date in dates:
    t = threading.Thread(target= download_for_data, args=(date,))
    threads.append(t)
    t.start()
    if len(threads) >= 10:
        for t in threads:
            t.join()
        threads = []

for t in threads:
    t.join()

In [ ]:
import requests
import xmltodict
import time
from saver import save_json
from utils import make_url, make_filename

def download_data(date, base):
    url = make_url(date, base)
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = xmltodict.parse(response.text)
            filename = make_filename(date, base)
            save_json(data, filename)
        else:
            print("Error getting data for", date)
    except:
        print("Problem getting data for", date)
    time.sleep(0.5)

In [ ]:
import os
import json

def save_json(data, filename):
    folder = os.path.dirname(filename)
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(filename, "w") as f:
        json.dump(data, f)

In [ ]:
import random
import os
from datetime import datetime, timedelta

rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY",
         "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR", "ILS", "KZT", "KRW",
         "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN",
         "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]

def pick_random_base():
    choices = []
    for r in rates:
        if r != "USD" and r != "EUR" and r != "GBP":
            choices.append(r)
    return random.choice(choices)

def get_all_dates(start):
    dates = []
    today = datetime.today()
    d = datetime.strptime(start, "%Y-%m-%d")
    while d <= today:
        dates.append(d.strftime("%Y-%m-%d"))
        d += timedelta(days=1)
    return dates

def make_url(date, base):
    return "https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date=" + date + "&base_currency_code=" + base + "&format_type=xml"

def make_filename(date, base):
    return "data/" + base + "/" + date + "_exchange_rates.json"

def already_downloaded(date, base):
    file_path = make_filename(date, base)
    return os.path.exists(file_path)

Quest 2 Attempt 1:

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json 

In [ ]:
def load_data(currency):
    folder = f"data"
    all_data = []

    if not os.path.exists(folder):
        return pd.DataFrame()
    
    for file in sorted(os.listdir(folder)):
        if file.endswith(".json"):
            with open(os.path.join(folder, file), "r") as f:
                data = json.load(f)
                print(json.dumps(data, indent=4))
                try:
                    items = data["exchangeRates"]["item"]
                    date = data["exchangeRates"]["@currencyDate"]
                    if isinstance(items, dict):
                        items = [items]
                    for rate in items:
                        to_code = rate["@currencyCode"]
                        value = float(rate["inverseRate"])
                        all_data.append({"date": date, "to": to_code, "rate": value})
                except Exception as e:
                    pass
    df = pd.DataFrame(all_data)
    if not df.empty:
        df["date"] = pd.to_datetime(df["date"])
    return df

In [ ]:
currencies = ["USD", "EUR", "GBP", "JPY", "CNY"]
all_frames = {}

for c in currencies:
    all_frames[c] = load_data(c)

In [ ]:
for currency, df in all_frames.items():
    if df.empty:
        print(f"No data for {currency}")
        continue

    top_currencies = df["to"].value_counts().head(3).index

    for to_curr in top_currencies:
        sub = df[df["to"] == to_curr]

        plt.figure(figsize=(10, 4))
        sns.lineplot(data=sub, x="date", y="rate")
        plt.title(f"{currency} to {to_curr} exchange rate over time")
        plt.xlabel("Date")
        plt.ylabel("Exchange Rate")
        plt.show()

Quest 2 Attempt 2: 

In [ ]:
import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

BASE_DIR = os.path.join(os.getcwd(), "Take Home Final/Data")

currencies = ["USD", "EUR", "GBP", "JPY", "CNY"]

def load_data(currency):
    folder = os.path.join(BASE_DIR, currency)
    all_data = []

    if not os.path.exists(folder):
        print(f"Folder missing for {currency}, skipping.")
        return pd.DataFrame()

    for file in sorted(os.listdir(folder)):
        if file.endswith(".json"):
            try:
                with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
                    data = json.load(f)

                if "exchange_rates" in data and "@date" in data["exchange_rates"]:
                    date = data["exchange_rates"]["@date"]
                    rate_info = data["exchange_rates"].get("exchange_rate", [])

                    if isinstance(rate_info, dict): 
                        rate_info = [rate_info]

                    for rate in rate_info:
                        if "@to_currency" in rate and "rate" in rate:
                            to_code = rate["@to_currency"]
                            value = float(rate["rate"])
                            all_data.append({"date": date, "to": to_code, "rate": value})
            except json.JSONDecodeError:
                print(f"Error reading {file}, skipping.")
                continue

    df = pd.DataFrame(all_data)

    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

    return df.dropna()

all_frames = {currency: load_data(currency) for currency in currencies}

for currency, df in all_frames.items():
    if df.empty:
        print(f"No data found for {currency}.")
    else:
        print(f"{currency}: Loaded {df.shape[0]} records.")

def plot_trends(data_frames):
    for currency, df in data_frames.items():
        if df.empty:
            continue

        top_currencies = df["to"].value_counts().head(3).index

        for to_curr in top_currencies:
            sub = df[df["to"] == to_curr]

            plt.figure(figsize=(10, 4))
            sns.lineplot(data=sub, x="date", y="rate")
            plt.title(f"{currency} to {to_curr} Exchange Rate Over Time")
            plt.xlabel("Date")
            plt.ylabel("Exchange Rate")
            plt.show()

def analyze_volatility(data_frames):
    for currency, df in data_frames.items():
        if df.empty:
            continue

        df = df.sort_values("date")
        df["change"] = df.groupby("to")["rate"].diff().abs()

        mean_volatility = df.groupby("to")["change"].mean().sort_values(ascending=False)
        print(f"Volatility for {currency}:")
        print(mean_volatility.head(3))
        print()

def plot_volatility(data_frames):
    for currency, df in data_frames.items():
        if df.empty:
            continue

        df["change"] = df.groupby("to")["rate"].diff().abs()

        plt.figure(figsize=(10, 4))
        sns.histplot(df["change"].dropna(), bins=50, kde=True)
        plt.title(f"{currency} Exchange Rate Volatility")
        plt.xlabel("Rate Change")
        plt.ylabel("Frequency")
        plt.show()

def plot_correlation(data_frames):
    valid_frames = [df for df in data_frames.values() if not df.empty]

    if not valid_frames:
        print("No valid data available for correlation analysis.")
        return

    combined_df = pd.concat(valid_frames, ignore_index=True)

    if "date" not in combined_df.columns:
        print("'date' column missing in combined dataset.")
        return

    combined_df["date"] = pd.to_datetime(combined_df["date"], errors="coerce")
    combined_df = combined_df.dropna(subset=["date"]) 

    combined_df = combined_df.groupby(["date", "to"], as_index=False).agg({"rate": "mean"})

    pivot_df = combined_df.pivot(index="date", columns="to", values="rate")

    if pivot_df.empty:
        print("No data available for correlation analysis after pivoting.")
        return

    plt.figure(figsize=(8, 6))
    sns.heatmap(pivot_df.corr(), annot=True, cmap="coolwarm")
    plt.title("Currency Correlations")
    plt.show()

analyze_volatility(all_frames)
plot_trends(all_frames)
plot_volatility(all_frames)
plot_correlation(all_frames)